# $S^{-}$ and $S^{+}$ Implementation

$$
S^{-} := \sum_{j=1}^n I^{\otimes(n-j)}\otimes\sigma_{01}\otimes\sigma_{10}^{\otimes(j-1)}
$$

$$
S^{+} := (S^-)^\dag = \sum_{j=1}^n I^{\otimes(n-j)}\otimes\sigma_{10}\otimes\sigma_{01}^{\otimes(j-1)}
$$

Primero se definirá las matrices sigmas y la identidad:

In [5]:
import numpy as np

# Define matrix sigma01
sigma_01 = np.array(
    [
        [0, 1],
        [0,0]
    ]
)

#define matrix sigma10
sigma_10 = np.array(
    [
        [0, 0],
        [1,0]
    ]
)

#define identity
identity = np.array(
    [
        [1, 0],
        [0,1]
    ]
)


Classiq tiene una función que pasa de matrices de formato `np.array` a hamiltoniano (Justo es lo que necesitamos). Entonces definiré el operador completo usando funciones que ofrece numpy

In [11]:
import numpy as np

def construct_s_minus(n):
    """
    Construye el operador S^- para n qubits.
    
    Args:
        n (int): Número de qubits.
    
    Returns:
        np.array: Matriz del operador S^- en el espacio de Hilbert de n qubits.
    """
    dim = 2 ** n  # Dimensión del espacio de Hilbert
    s_minus = np.zeros((dim, dim), dtype=complex)  # Inicializar el operador en ceros

    # Sumar los términos de la definición
    for j in range(1, n + 1):
        # Construir las partes del producto tensorial
        if n - j > 0:
            left_identity = np.eye(2 ** (n - j))
        else:
            left_identity = 1  # Escalar cuando no hay identidad a la izquierda

        if j - 1 > 0:
            right_sigma_10 = sigma_10
            for _ in range(j - 2):  # Hacer producto tensorial de σ_{10}^{⊗(j-1)}
                right_sigma_10 = np.kron(right_sigma_10, sigma_10)
        else:
            right_sigma_10 = 1  # Escalar cuando no hay σ_{10} a la derecha

        # Término tensorial: I^{\otimes(n-j)} ⊗ σ_{01} ⊗ σ_{10}^{⊗(j-1)}
        term = np.kron(
            np.kron(left_identity, sigma_01),
            right_sigma_10
        )

        # Sumar este término al operador S^-
        s_minus += term

    return s_minus

# Ejemplo: Construir el operador S^- para n = 1 (deberia dar sigma 01)
n = 1
s_minus_operator = construct_s_minus(n)

# Mostrar el resultado
print("Operador S^- (n = {}):".format(n))
print(s_minus_operator)

# Verificar el tipo de dato
print("\nTipo de dato:", type(s_minus_operator))


Operador S^- (n = 1):
[[0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j]]

Tipo de dato: <class 'numpy.ndarray'>


La función que pasa de matrices a hamiltonianos en Classiq es `matrix_to_hamiltonian`, realmente nose como funciona pero recomiendo usarla como caja negra

In [14]:
from classiq import *

S_minus = construct_s_minus(2)
S_plus = S_minus.conj().T

hamiltonian_S_minus = matrix_to_hamiltonian(S_minus, is_hermitian=False)
hamiltonian_S_plus = matrix_to_hamiltonian(S_plus, is_hermitian=False)
